![iscap_logo](https://www.iscap.ipp.pt/logo-ipp.png)

# Trabalho Prático

O objetivo do trabalho é desenvolver um programa para fazer a gestão de alunso numa instituição de ensino. O programa não necessita de ter interface gráfica, podendo apenas funcionar à base de inputs como implementado no código de exemplo.

### As funcionalidades obrigatórias são:
* Permitir visualizar todos os alunos
* Permitir criar novos alunos por upload de csv (exemplo fornecido) e via input()
* Permitir editar / apagar alunos
* Permitir exportar os dados de alunos para excel
* Permitir adicionar / apagar unidades curriculares
* Visualizar estatisticas básicas como:
    * Número de alunos
    * Mínimo, média, máximo de nota (geral e por unidade curricular)
    * Número de alunos aprovados (geral e por unidade curricular
    * ...
* Bónus: implementar pelo menos mais uma funcionalidade extra, idealizada pelo grupo. Máximo de 1.5 valores extra para funcionalidades criativas e com complexidade adicional.

### Funcionamento base:
* Sempre que o programa é iniciado (correndo a função 'run') deverá verificar se tem um ficheiro no sistema com o nome "database.csv", se sim deverá ler o ficheiro e assumir esses registos por default.
* Ao sair do programa, este deverá questionar o user se quer guardar as alterações, se sim o programa deverá fazer overwrite do ficheiro "database.csv".
* Ao inserir novos alunos deverá apenas aceitar caso os dados os nomes estejam na lista de novos possiveis (permitido usar a função 'fetch_list_from_txt'). Além disso, os nomes devem ser sempre guardados com a primeira letra em maiúsculas e as restantes em minúsculas. As restantes colunas devem ser todas convertidas para minúsculas. Estas condições são válidas tanto para a inserção manual como via ficheiro.
* Internamente a representação da informação dos aluns deverá ser guardada num pandas DataFrame.

### Identificação do grupo
| nome | número aluno |
|------|--------------|
|Ana   | 2130248      |
|David | 2242131      |
|João  | 2240515      |
|      |              |

In [5]:
import pandas as pd
import requests
from os.path import exists
import random

# URLs dos arquivos com nomes
FILE_FIRST_NAMES = "https://raw.githubusercontent.com/Tremocex/ALL_NAMES_PORTUGAL/main/names_final.txt"
FILE_SURNAMES = "https://raw.githubusercontent.com/Tremocex/ALL_NAMES_PORTUGAL/main/surnames_final.txt"

# Funções Úteis
def fetch_list_from_txt(url: str) -> list:
    txt = requests.get(url).text
    return list(set(txt.split("\n")))

# Carregar nomes e sobrenomes permitidos
FIRST_NAMES = fetch_list_from_txt(FILE_FIRST_NAMES)
SURNAMES = fetch_list_from_txt(FILE_SURNAMES)

# Verificar a existência do arquivo database_example.csv
def exists_database(file_path: str = "database_example.csv") -> bool:
    return exists(file_path)

# Carregar o arquivo CSV para um DataFrame (se existir)
def load_data(file_path: str = "database_example.csv"):
    if exists_database(file_path):
        return pd.read_csv(file_path)
    else:
        return pd.DataFrame(columns=["primeiro nome", "último nome", "unidade curricular", "nota", "student_id", "data de nascimento", "cidade"])

# Guardar o DataFrame de volta no arquivo CSV
def save_data(df, file_path: str = "database_example.csv"):
    df.to_csv(file_path, index=False)

# Função para gerar um student ID aleatório
def generate_student_id() -> str:
    return f"2024{random.randint(1000, 9999)}"

# Função para criar um novo aluno
def create_new_student():
    first_name = input("Digite o primeiro nome: ").strip().lower()
    surname = input("Digite o sobrenome: ").strip().lower()

    # Verificar se o nome e sobrenome estão na lista de nomes válidos
    if first_name not in [name.lower() for name in FIRST_NAMES]:
        print(f"Primeiro nome '{first_name}' não encontrado na lista de nomes.")
        return
    if surname not in [s.lower() for s in SURNAMES]:
        print(f"Sobrenome '{surname}' não encontrado na lista de sobrenomes.")
        return

    # Gerar um student ID
    student_id = generate_student_id()

    # Adicionar dados adicionais: cidade e data de nascimento
    city = input("Digite a cidade: ").strip().capitalize()
    birthday = input("Digite a data de nascimento (dd/mm/yyyy): ").strip()

    # Adicionar as unidades curriculares e suas respectivas notas
    ucs = []
    grades = []

    print("Digite as unidades curriculares e as respectivas notas. Deixe em branco para terminar.")
    
    while True:
        uc = input("Unidade curricular: ").strip()
        if not uc:
            break
        grade = input(f"Nota para {uc}: ").strip()
        try:
            grade = float(grade)
            if grade < 0 or grade > 20:
                print("Nota inválida. Digite uma nota entre 0 e 20.")
                continue
        except ValueError:
            print("Nota inválida. Digite um número válido.")
            continue
        
        ucs.append(uc)
        grades.append(grade)

    # Criar registros para cada unidade curricular
    all_students = []
    for uc, grade in zip(ucs, grades):
        student = {
            "primeiro nome": first_name.capitalize(),
            "último nome": surname.capitalize(),
            "unidade curricular": uc,
            "nota": grade,
            "student_id": student_id,
            "data de nascimento": birthday,
            "cidade": city
        }
        all_students.append(student)

    # Salvar os dados no DataFrame
    if all_students:
        df = load_data()
        new_df = pd.DataFrame(all_students)
        df = pd.concat([df, new_df], ignore_index=True)
        save_data(df)
        print("Novo aluno adicionado com sucesso!")

# Função para carregar alunos em lote via CSV e mapear corretamente as unidades curriculares
def load_batch_students():
    file_path = input("Digite o caminho do arquivo CSV com os alunos: ").strip()
    try:
        new_data = pd.read_csv(file_path)

        # Mapear os dados
        all_students = []

        # Iterar sobre cada linha do CSV e organizar os dados
        for _, row in new_data.iterrows():
            first_name = row['first_name'].strip().lower()
            surname = row['surname'].strip().lower()

            # Verificar se o primeiro nome e o sobrenome são válidos
            if first_name not in [name.lower() for name in FIRST_NAMES]:
                print(f"Primeiro nome '{first_name}' não encontrado na lista de nomes.")
                continue
            if surname not in [s.lower() for s in SURNAMES]:
                print(f"Último nome '{surname}' não encontrado na lista de últimos nomes.")
                continue

            # Gerar o student_id
            student_id = generate_student_id()

            # Obter as unidades curriculares e notas
            ucs = []
            grades = []

            # Mapear as unidades curriculares e notas
            for uc_col, grade_col in [
                ('analise_dados', 'analise_dados_grade'),
                ('bi', 'bi_grade'),
                ('fpp', 'fpp_grade'),
                ('ml', 'ml_grade'),
                ('cloud', 'cloud_grade'),
                ('deep_learning', 'deep_learning_grade'),
                ('nlp', 'nlp_grade'),
                ('time_series', 'time_series_grade')
            ]:
                if grade_col in row and not pd.isna(row[grade_col]):
                    ucs.append(uc_col)
                    grades.append(float(row[grade_col]))

            # Se o aluno tiver unidades curriculares e notas
            if ucs and grades:
                # Para cada unidade curricular, criar uma linha separada
                for uc, grade in zip(ucs, grades):
                    student = {
                        "primeiro nome": first_name.capitalize(),
                        "último nome": surname.capitalize(),
                        "unidade curricular": uc,
                        "nota": grade,
                        "student_id": student_id,
                        "data de nascimento": row.get('birthday', 'Desconhecida'),
                        "cidade": row.get('city', 'Desconhecida')
                    }
                    all_students.append(student)

        # Adicionar os novos alunos ao DataFrame
        if all_students:
            df = load_data()
            new_df = pd.DataFrame(all_students)
            df = pd.concat([df, new_df], ignore_index=True)
            save_data(df)
            print("Alunos carregados com sucesso!")

    except Exception as e:
        print(f"Erro ao carregar os alunos: {e}")

# Função para exibir alunos atuais de forma compacta
def show_current_students():
    df = load_data()
    if df.empty:
        print("Não há alunos registados.")
        return

    # Criar uma tabela para exibir cada aluno com as UCs como colunas
    df_pivot = df.pivot_table(index=["primeiro nome", "último nome", "student_id", "data de nascimento", "cidade"], 
                              columns="unidade curricular", 
                              values="nota", 
                              aggfunc="first")

    # Reset no índice para ter uma visualização mais limpa
    df_pivot = df_pivot.reset_index()

    # Exibir o DataFrame formatado
    print(df_pivot)

# Função para editar ou apagar registos de alunos
def change_record():
    df = load_data()
    if df.empty:
        print("Não há alunos registados.")
        return

    # Criar uma tabela para exibir cada aluno com as UCs como colunas
    df_pivot = df.pivot_table(index=["primeiro nome", "último nome", "student_id"], 
                              columns="unidade curricular", 
                              values="nota", 
                              aggfunc="first")  

    # Reset no índice para ter uma visualização mais limpa
    df_pivot = df_pivot.reset_index()

    print("Escolha o aluno que deseja editar/apagar:")
    print(df_pivot[["primeiro nome", "último nome", "student_id"] + [col for col in df_pivot.columns if col not in ["primeiro nome", "último nome", "student_id"]]])

    student_index = int(input("Digite o índice do aluno (começando de 0) ou -1 para cancelar: ").strip())

    if student_index == -1:
        return

    if student_index < 0 or student_index >= len(df_pivot):
        print("Índice inválido!")
        return

    # Exibir o aluno selecionado
    selected_student = df_pivot.iloc[student_index]
    print(f"Aluno selecionado: {selected_student['primeiro nome']} {selected_student['último nome']} - ID: {selected_student['student_id']}")
    for uc in selected_student.index:
        if uc not in ["primeiro nome", "último nome", "student_id"]:
            print(f"Unidade curricular: {uc} - Nota: {selected_student[uc]}")

    # Perguntar ao utilizador se quer editar ou apagar
    action = input("Digite 'e' para editar, 'd' para apagar o aluno, ou '-1' para cancelar: ").strip().lower()

    if action == "e":
        # Editar unidades curriculares e notas
        uc = input("Digite a unidade curricular que deseja editar: ").strip()
        if uc not in selected_student.index:
            print(f"Unidade curricular '{uc}' não encontrada.")
            return
        try:
            grade = float(input(f"Digite a nova nota para a unidade curricular '{uc}': "))
        except ValueError:
            print("Nota inválida!")
            return

        # Atualizar a nota no DataFrame original
        df.loc[(df["primeiro nome"] == selected_student["primeiro nome"]) & 
               (df["último nome"] == selected_student["último nome"]) & 
               (df["student_id"] == selected_student["student_id"]) & 
               (df["unidade curricular"] == uc), "nota"] = grade

        # Guardar alterações
        save_data(df)
        print("Dados do aluno atualizados com sucesso!")

    elif action == "d":
        # Apagar o aluno
        df = df.drop(df[(df["primeiro nome"] == selected_student["primeiro nome"]) & 
                        (df["último nome"] == selected_student["último nome"]) & 
                        (df["student_id"] == selected_student["student_id"])].index)
        save_data(df)
        print("Aluno apagado com sucesso!")

    elif action == "-1":
        # Cancelar ação
        print("Operação cancelada.")
        return

    else:
        print("Comando inválido!")
        return
# Função para exibir estatísticas sobre os alunos, incluindo aprovados e reprovados por UC
def show_stats():
    df = load_data()
    if df.empty:
        print("Não há alunos registados.")
        return

    total_students = len(df)
    min_grade = df["nota"].min()
    max_grade = df["nota"].max()
    avg_grade = df["nota"].mean()
    passed_students = len(df[df["nota"] >= 10])
    failed_students = total_students - passed_students

    print(f"Total de alunos: {total_students}; Total de alunos aprovados: {passed_students}; Total de alunos reprovados: {failed_students}.")
    print(f"Nota mínima: {min_grade}")
    print(f"Nota máxima: {max_grade}")
    print(f"Nota média: {avg_grade:.2f}")

    # Estatísticas gerais por unidade curricular
    units_stats = df.groupby("unidade curricular").agg(
        min_grade=('nota', 'min'),
        max_grade=('nota', 'max'),
        avg_grade=('nota', 'mean'),
        count_students=('nota', 'count')
    ).reset_index()

    # Adicionar contagem de aprovados e reprovados
    units_stats['approved'] = df[df['nota'] >= 10].groupby('unidade curricular')['nota'].count().values
    units_stats['failed'] = units_stats['count_students'] - units_stats['approved']

    print("\nEstatísticas por unidade curricular:")
    print(units_stats)

    return units_stats

# Função para exportar estatísticas para Excel ou CSV
def export_stats():
    stats = show_stats()
    if stats is None:
        return

    # Perguntar ao utilizador onde guardar o arquivo
    file_path = input("Digite o nome do arquivo para exportar (exemplo: estatisticas.xlsx ou estatisticas.csv): ").strip()

    # Guardar as estatísticas em Excel ou CSV conforme o nome do arquivo
    if file_path.endswith(".xlsx"):
        stats.to_excel(file_path, index=False)
    elif file_path.endswith(".csv"):
        stats.to_csv(file_path, index=False)
    else:
        print("Formato de arquivo não suportado. Use .xlsx ou .csv.")
        return

    print(f"Estatísticas exportadas para {file_path} com sucesso!")

# Função para exportar dados para Excel com UCs como colunas, incluindo cidade e data de nascimento
def export_data():
    df = load_data()
    if df.empty:
        print("Não há dados para exportar.")
        return

    # Criar uma tabela para exibir cada aluno com as UCs, cidade e data de nascimento como colunas
    df_pivot = df.pivot_table(index=["primeiro nome", "último nome", "student_id", "data de nascimento", "cidade"], 
                              columns="unidade curricular", 
                              values="nota", 
                              aggfunc="first") 

    # Reset no índice para ter uma visualização mais limpa
    df_pivot = df_pivot.reset_index()

    # Perguntar ao utilizador onde guardar o arquivo
    file_path = input("Digite o nome do arquivo Excel (exemplo: alunos.xlsx): ").strip()

    # Salvar o DataFrame como Excel
    df_pivot.to_excel(file_path, index=False)
    print(f"Dados exportados para {file_path} com sucesso!")

# Função para sair do programa
def exit_program():
    print("Deseja guardar as alterações? (s/n): ", end="")
    if input().strip().lower() == "s":
        df = load_data()
        save_data(df)
    print("A sair do programa. Até logo!")

# Menu do programa
menu = {
    "1": ("Create new student", create_new_student),
    "2": ("Load batch students (csv file)", load_batch_students),
    "3": ("Show current students", show_current_students),
    "4": ("Change record", change_record),
    "5": ("Show stats about current class", show_stats),
    "6": ("Export stats", export_stats),
    "7": ("Download data", export_data),
    "-1": ("Exit", exit_program)
}

# Função principal do programa
def run(menu):
    while True:
        for cmd, (cmd_msg, _) in menu.items():
            print(f"{cmd} - {cmd_msg}")

        cmd = input()
        if cmd not in menu.keys():
            print("Comando inválido")
            continue
        menu[cmd][1]()

        if cmd == "-1":
            break

run(menu)



1 - Create new student
2 - Load batch students (csv file)
3 - Show current students
4 - Change record
5 - Show stats about current class
6 - Export stats
7 - Download data
-1 - Exit


 -1


Deseja guardar as alterações? (s/n): 

 n


A sair do programa. Até logo!
